##

In [1]:
from matplotlib import pyplot as plt
import numpy as np
import torch

import torchvision
from torchvision import datasets, transforms

from collections import Counter
from torch.utils.data import random_split

In [2]:
seed = 10
torch.manual_seed(seed)

category_index = 8
n_val = 5000

data_path = '/cifar-10-batches-py'

Function for loading the Cifar10 dataset.

The method will have to be run twice.
After running the method for the first time we get create a normalizer from the std and mean of the images. The method is then ran for a second time with the normalizer as the preprocessor.

Loading the CIFAR-10 dataset as tensors.

In [3]:
transformed_cifar10_train_val = datasets.CIFAR10(
    data_path,
    train=True,
    download=True,
    transform = transforms.ToTensor()
)

Files already downloaded and verified


Stacking the set of images into a single tensor. We then create a normalizer for the dataset around the mean and standard deviation of the 3 dimensions (height, width channel (color)).

In [4]:
imgs = torch.stack([img for img, _ in transformed_cifar10_train_val])

normalizer = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean = imgs.mean(dim=(0, 2, 3)), std = imgs.std(dim=(0, 2, 3)))])

Loading the dataset as tensors for training+validation and testing. This time we apply the composition of transforms.

In [5]:
normalized_cifar10_train_val = datasets.CIFAR10(
    data_path,
    train=True,
    download=False,
    transform = normalizer
)


transformed_cifar10_test = datasets.CIFAR10(
    data_path,
    train=False,
    download=False,
    transform = normalizer
)

As this is a binary classification problem where we only want to identify whether an image is a ship or not, we can set the labels that are "ship" to true. We set all other labels to false.

In [6]:
train_labels = np.array([label for _, label in transformed_cifar10_train_val])
train_labels = np.array(train_labels == category_index).astype(int)

test_labels = np.array([label for _, label in transformed_cifar10_test])
test_labels = np.array(test_labels == category_index).astype(int)

Splitting the training and validation set randomly.

In [7]:
n_train = len(transformed_cifar10_train_val)-n_val

transformed_cifar10_train_split, transformed_cifar10_val_split = random_split(
    transformed_cifar10_train_val,
    [n_train, n_val],

    generator=torch.Generator().manual_seed(seed)
)

print("Size of the train dataset:        ", len(transformed_cifar10_train_split))
print("Size of the validation dataset:   ", len(transformed_cifar10_val_split))
print("Size of the test dataset:         ", len(transformed_cifar10_test))

Counter([label for _, label in transformed_cifar10_train_split])

Size of the train dataset:         45000
Size of the validation dataset:    5000
Size of the test dataset:          10000


Counter({8: 4486,
         7: 4486,
         1: 4512,
         6: 4510,
         3: 4516,
         5: 4487,
         4: 4499,
         2: 4483,
         0: 4518,
         9: 4503})

Choosing a pre-trained CNN model: we chose ResNet18, which is not trained on Cifar-10.

In [8]:
from torchvision import models
import torch.nn as nn
import torch.optim as optim

Loading pre-trained ResNet18 model and modifying the last layer. We are doing binary classification, so we think we only need one node in the final layer.

In [9]:
model = models.resnet18(weights='ResNet18_Weights.DEFAULT')

num_features = model.fc.in_features
model.fc = nn.Linear(num_features,1)

We use Binary Cross Entropy as it should be suitable for binary classification problems (add reasoning and explanation). We use nn.BCEWithLogitsLoss() as it combines the sigmoid activation function and the BCE into a single class.

The optimizer we use is Adam, and we will begin with a learning rate of 0.001, just because it is a commonly used learning rate.

In [10]:
loss_function = nn.BCEWithLogitsLoss()

optimizer = optim.Adam(model.parameters(),  lr=0.001)

In [26]:
from torch.utils.data import DataLoader
import torch.optim.lr_scheduler as lr_scheduler
import time

train_loader = DataLoader(transformed_cifar10_train_split, batch_size=64, shuffle=False)
val_loader = DataLoader(transformed_cifar10_val_split, batch_size=64, shuffle=False)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

num_epochs = 3

model.train()

for epoch in range(num_epochs):
    start_time = time.time()
    running_loss = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        running_loss += loss.item()
    end_time = time.time()
    epoch_time = end_time - start_time
    print(f"Epoch {epoch + 1}, Time: {epoch_time:.2f} seconds, Loss: {running_loss / len(train_loader)}")
    
'''
model.eval()
val_loss = 0.0
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs).squeeze()
        loss = loss_function(outputs, labels.float())
        
        val_loss += loss.item()

        
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pth')
    
'''

    


Epoch 1, Time: 487.74 seconds, Loss: -4749724.256392046


KeyboardInterrupt: 

In [19]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate(model, data_loader):
    model.eval()
    y_true = []
    y_pred = []

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs).squeeze()
            predictions = (torch.sigmoid(outputs) > 0.5).int()

            y_true.extend(labels.tolist())
            y_pred.extend(predictions.tolist())

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, pos_label=1, average='binary')
    recall = recall_score(y_true, y_pred, pos_label=1, average='binary')
    f1 = f1_score(y_true, y_pred, pos_label=1, average='binary')

    return accuracy, precision, recall, f1


In [23]:
# Here we use shuffle = False
# Because it is easier to check the predictions made.
train_loader = torch.utils.data.DataLoader(transformed_cifar10_train_split, batch_size=64, shuffle=False)
val_loader = torch.utils.data.DataLoader(transformed_cifar10_val_split, batch_size=64, shuffle=False)

def compute_accuracy(model, loader):
    model.eval()
    correct = 0
    total = 0

    # We do not want gradients here, as we will not want to update the parameters.
    with torch.no_grad():
        for imgs, labels in loader:

            outputs = model(imgs)
            _, predicted = torch.max(outputs, dim=1)
            total += labels.shape[0]
            correct += int((predicted == labels).sum())

    acc =  correct / total
    print("Accuracy: {:.2f}".format(acc))
    return acc

print("Training accuracy:")
compute_accuracy(model, train_loader)
print("Validation accuracy:")
compute_accuracy(model, val_loader)


Training accuracy:
Accuracy: 0.10
Validation accuracy:
Accuracy: 0.10


0.0964

In [22]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate(model, data_loader):
    model.eval()
    y_true = []
    y_pred = []
    
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs).squeeze()
            predictions = (torch.sigmoid(outputs) > 0.5).int()
            
            y_true.extend(labels.tolist())
            y_pred.extend(predictions.tolist())
    
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    
    return accuracy, precision, recall, f1

    

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate(model, data_loader):
    model.eval()
    y_true = []
    y_pred = []

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs).squeeze()
            predictions = (torch.sigmoid(outputs) > 0.5).int()

            y_true.extend(labels.tolist())
            y_pred.extend(predictions.tolist())

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, pos_label=1, average='binary')
    recall = recall_score(y_true, y_pred, pos_label=1, average='binary')
    f1 = f1_score(y_true, y_pred, pos_label=1, average='binary')

    return accuracy, precision, recall, f1

test_loader = DataLoader(transformed_cifar10_test, batch_size=32, shuffle=False)
model.load_state_dict(torch.load('best_model.pth'))
accuracy, precision, recall, f1 = evaluate(model, test_loader)

print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1 Score: {f1}")


ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].